# MDL DB roll decay evaluation

In [29]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import os

import data
import copy
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Base, Model, LoadingCondition, Run, Min, Mean, Max, Std
from mdldb import mdl_to_evaluation
from evaluation.run_dynamic import RunDynamic
from evaluation.run_manoeuvring import RunZigZag

from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.norwegian_estimator import NorwegianEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
#from rolldecay.equations_lambdify import calculate_acceleration, calculate_velocity
from sklearn.pipeline import Pipeline


In [31]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

In [32]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=0.4)
scaler = ScaleFactorTransformer(scale_factor=None)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(15), phi_min=np.deg2rad(1))
offset_transformer = OffsetTransformer()
bounds = {
    'zeta':(0,np.inf),  # Only positive damping
    'd':(0,np.inf),  # Only positive damping
}

## Linear method

In [33]:
direct_linear_estimator = DirectLinearEstimator()

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('norwegian_estimator', direct_linear_estimator)]
        
pipeline_direct_linear = Pipeline(steps) # define the pipeline object.

In [37]:
def fit_predict(db_run, pipeline):
    
    ascii_file = db_run.load()
    df_raw = ascii_file.channels
    df = mdl_to_evaluation.do_transforms(df=df_raw)
    _pipline = copy.deepcopy(pipeline)
    
    scaler = _pipline['scaler']
    scaler.scale_factor = db_run.model.scale_factor
    
    df.rename(columns={'MA/Roll':'phi'}, inplace=True)
    
    # Fit:
    _pipline.fit(X=df)
    
    # Predict:
    estimator = _pipline[-1]
    s = {} 
    s.update(estimator.parameters)
    s['score'] = estimator.score(X=estimator.X)
    s['mean_damping'] = estimator.calculate_average_linear_damping()
    
    return s
    
    

In [38]:
roll_decay_tests = db.session.query(Run).filter(Run.test_type=='roll decay')

## Add roll decay

In [39]:
db_run = roll_decay_tests[0]
fit_predict(db_run=db_run, pipeline=pipeline_direct_linear)


AssertionError: 

In [23]:
roll_decay_tests = db.session.query(Run).filter(Run.test_type=='roll decay')

for db_run in roll_decay_tests:
    
    if db_run.ship_speed is None:
        db_run.ship_speed=0
    
    run = mdl_to_evaluation.db_run_to_evaluation_run(run=db_run)
    run.evaluate()    
    
    interesting_columns = ['x0','y0','z0','u','v','roll','pitch','psi','p','q','r','delta']
    statistics = run.results['statistics'][interesting_columns]
    
    min = statistics.loc['min']
    mean = statistics.loc['mean']
    max = statistics.loc['max']
    std = statistics.loc['std']
    
    min_db = Min(run_id=db_run.id,**min)
    db.session.merge(min_db)
    #db.session.commit()
    
    mean_db = Mean(run_id=db_run.id,**mean)
    db.session.merge(mean_db)
    #db.session.commit()
    
    max_db = Max(run_id=db_run.id,**max)
    db.session.merge(max_db)
    #db.session.commit()
    
    std_db = Std(run_id=db_run.id,**std)
    db.session.merge(std_db)
    db.session.commit()

In [24]:
db_run.loading_condition